# Self-Attention Mechanism: From Scratch

This notebook implements the self-attention mechanism from scratch using PyTorch.

## Table of Contents
1. [Introduction](#introduction)
2. [Mathematical Foundation](#math)
3. [PyTorch Implementation](#pytorch)


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")


ModuleNotFoundError: No module named 'torch'

## 1. Introduction {#introduction}

### How we represent the attention

Attention should be calculated from an input -> target processed with a weight. Take an example, if we concat user embedding and an item embedding, and map to how a user could relate to an item. The output we are looking for will be the attention for user -> item.

### Now, let's talk about self-attention

Self attention is a unique condition, where the input and target is the same object.

Take a look at following formula:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Let's understand this step by step:

1. $QK^T$ will calculate the heatmap of how x focuses on itself:
   ```
   X    a    b    c
   a    2    1    4
   b    3    3    1
   c    8    1    2
   ```

2. Softmax will apply a normalization for this table, and make sum of all elements into 1

3. For each attention after softmax, we process another dot product, as we discussed above, dot product can represent internal relationship between two things.

4. After all these processes, we are going to have an attention tensor, the higher score we have, it means the higher relation they have.

### Key Concepts:
- **Query (Q)**: What am I looking for?
- **Key (K)**: What do I have to offer?
- **Value (V)**: What is the actual content?

In self-attention, Q, K, and V are all derived from the same input sequence.


## 2. Mathematical Foundation {#math}

The self-attention mechanism can be expressed as:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Where:
- $Q$: Query matrix (n × d_k)
- $K$: Key matrix (n × d_k) 
- $V$: Value matrix (n × d_v)
- $d_k$: Dimension of key vectors
- $\sqrt{d_k}$: Scaling factor to prevent vanishing gradients

### Step-by-step process:
1. Compute attention scores: $QK^T$
2. Scale by $\sqrt{d_k}$
3. Apply softmax to get attention weights
4. Multiply by values: $\text{softmax}(\cdot)V$


## 3. PyTorch Implementation {#pytorch}

Let's implement self-attention using PyTorch.


In [ ]:
class SelfAttention(nn.Module):
    """Self-attention mechanism implementation in PyTorch."""
    
    def __init__(self, d_model: int, d_k: int = None, d_v: int = None):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_k if d_k is not None else d_model
        self.d_v = d_v if d_v is not None else d_model
        
        # Linear transformations for Q, K, V
        self.W_q = nn.Linear(d_model, self.d_k, bias=False)
        self.W_k = nn.Linear(d_model, self.d_k, bias=False)
        self.W_v = nn.Linear(d_model, self.d_v, bias=False)
        
        # Output projection
        self.W_o = nn.Linear(self.d_v, d_model, bias=False)
        
    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass of self-attention.
        
        Args:
            x: Input tensor (batch_size, seq_len, d_model)
            
        Returns:
            output: Attention output (batch_size, seq_len, d_model)
            attention_weights: Attention weights (batch_size, seq_len, seq_len)
        """
        batch_size, seq_len, _ = x.shape
        
        # Compute Q, K, V
        Q = self.W_q(x)  # (batch_size, seq_len, d_k)
        K = self.W_k(x)  # (batch_size, seq_len, d_k)
        V = self.W_v(x)  # (batch_size, seq_len, d_v)
        
        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1))  # (batch_size, seq_len, seq_len)
        scores = scores / np.sqrt(self.d_k)
        
        # Apply softmax
        attention_weights = F.softmax(scores, dim=-1)
        
        # Apply attention to values
        attended_values = torch.matmul(attention_weights, V)  # (batch_size, seq_len, d_v)
        
        # Output projection
        output = self.W_o(attended_values)  # (batch_size, seq_len, d_model)
        
        return output, attention_weights


In [ ]:
# Test the implementation
d_model = 8
seq_len = 4
batch_size = 1

# Create sample input
x = torch.randn(batch_size, seq_len, d_model)

# Initialize attention layer
attention = SelfAttention(d_model)

# Forward pass
output, attention_weights = attention(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attention_weights.shape}")
print(f"\nAttention weights (first sample):")
print(attention_weights[0].detach().numpy())


In [ ]:
# Test with a longer sequence
seq_len = 8
x_long = torch.randn(batch_size, seq_len, d_model)
output_long, attention_weights_long = attention(x_long)

print(f"Longer sequence attention weights shape: {attention_weights_long.shape}")
print(f"Attention weights (first sample):")
print(attention_weights_long[0].detach().numpy())


In this notebook, we implemented the self-attention mechanism from scratch using PyTorch:

1. **Mathematical Foundation**: Understanding the attention formula and scaling
2. **PyTorch Implementation**: Complete self-attention layer with proper linear transformations

### Key Takeaways:
- Self-attention allows models to focus on relevant parts of the input
- The mechanism is highly parallelizable and interpretable
- Attention weights reveal what the model is "looking at"
- The scaling factor $\sqrt{d_k}$ prevents vanishing gradients

This foundation is essential for understanding more complex architectures like Transformers!
